In [1]:
import psycopg2

import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [2]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [3]:
water_df = pd.read_sql("select sample_code, batch_date, analysis_name, chemical_name, result, unit_name from SampleResults where group_name='Water Analysis'",con=conn_lims)
# batch_date > '2024-05-21 09:32:00.000' AND

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
water_df.to_csv("input/water_samples.csv")

In [5]:
water_df = pd.read_csv("input/water_samples.csv")

In [6]:
len(water_df['analysis_name'].unique())

1014

In [7]:
water_df

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,pH,8.27,NaN
1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,pH,8.25,NaN
2,2,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",pH,7.25,NaN
3,3,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",Electrical Conductivity,0.15,mS cm -1
4,4,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",Ammonium,< 0.01,ppm
...,...,...,...,...,...,...,...
1208496,1208496,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),Nitrate N,5.90,ppm
1208497,1208497,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),Phosphorus,0.012,ppm
1208498,1208498,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),Sulphur,5.27,ppm
1208499,1208499,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),Iron,0.013,ppm


In [8]:
water_df['analysis_name'].describe()

count                                    1208501
unique                                      1014
top       Standard Drinking Water Analysis (WHO)
freq                                      175578
Name: analysis_name, dtype: object

In [9]:
water_df = water_df.loc[~(water_df['result'] == "ND") ]
water_df = water_df.loc[~(water_df['result'] == "Detected") ]

In [10]:
water_df = water_df.dropna(subset="result")

In [11]:
water_df

,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,pH,8.27,NaN
1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,pH,8.25,NaN
2,2,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",pH,7.25,NaN
3,3,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",Electrical Conductivity,0.15,mS cm -1
4,4,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",Ammonium,< 0.01,ppm
...,...,...,...,...,...,...,...
1208496,1208496,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),Nitrate N,5.90,ppm
1208497,1208497,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),Phosphorus,0.012,ppm
1208498,1208498,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),Sulphur,5.27,ppm
1208499,1208499,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),Iron,0.013,ppm


In [12]:
water_df['result'] = [ float(i.replace("<","").replace(">","").strip()) for i in water_df['result']]

In [13]:
water_df['unit_name'] = [ str(i).strip() for i in water_df['unit_name'].values ]

In [14]:
water_df['chemical_name'] = [ str(i).lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in water_df['chemical_name'].values ]

In [15]:
water_df = water_df.loc[~(water_df['result'] < 0)]

In [16]:
water_df = water_df.loc[~(water_df['unit_name'] == "nan")]

In [18]:
df_ = water_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/water_unit_per_chemical.csv")

In [19]:
df_ = df_.reset_index()

In [20]:
crop_chem_unit_df = pd.DataFrame()
for crop in df_['crop_name'].unique():
    print(crop)
    chems_count_ = df_.loc[df_['crop_name'] == crop]
    duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
    temp_crop_chem_unit_df = pd.DataFrame()
    for index,row in duplicates_chem.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"crop":[crop],"chemical_name":[chemical], "unit_name":[unit]})
        temp_crop_chem_unit_df = pd.concat([temp_crop_chem_unit_df, _])
    crop_chem_unit_df = pd.concat([crop_chem_unit_df, temp_crop_chem_unit_df])
crop_chem_unit_df.to_csv("output/water_unit_per_chemical_decision.csv")

KeyError: 'crop_name'

In [ ]:
water_df_copy = water_df.copy(deep=True)

In [ ]:
for index,row in crop_chem_unit_df.iterrows():
    crop = row.crop
    chemical = row.chemical_name
    unit = row.unit_name
    print(row)

    water_df_copy = (water_df_copy.loc[~((water_df_copy['crop_name'] == crop) & (water_df_copy['chemical_name'] == chemical) & (water_df_copy['unit_name'] != unit))])

In [ ]:
water_df

In [ ]:
water_df_copy

In [ ]:
for crop in water_df['crop_name'].unique():
    crop_df = water_df.loc[water_df['crop_name'] == crop]
    print(crop_df)
    break

In [ ]:
[ i for i in water_df.columns]

In [ ]:
water_df.to_csv("input/water_analysis_cleaned.csv")

In [ ]:
params_count_dict = {}
params_dict = {}

In [ ]:
for i in water_df['crop_name'].unique():
    print(i)
    temp = water_df.loc[water_df['crop_name'] == i].pivot_table(columns="chemical_name", index="sample_code", values="result")
    temp_count = pd.DataFrame(temp.isna().sum(axis=0))

    print(len(temp_count[temp_count != len(temp)].index))
    params_count_dict[i] = temp.isna().sum(axis=0).to_dict()
    params_dict[i] = temp_count[temp_count != len(temp)].index

In [ ]:
pd.DataFrame(params_count_dict).to_csv("output/parameters_count_per_crop_name.csv")

In [ ]:
df = pd.DataFrame()
for crop, params in params_dict.items():
    _ = pd.DataFrame(columns=[i for i in params])
    for col in _.columns:
        _[col] = [True]
    _['crop_name'] = crop
    df = pd.concat([df,_])
df.to_csv("output/parameters_per_crop.csv")